In [1]:
# wide extend of jupyter
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%load_ext autoreload
%autoreload 2
from IPython.core.display import Image, display

import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from IPython.core.display import Image, display, HTML
import pandas as pd
from tqdm import tqdm
tqdm.pandas(desc='progress')
pd.set_option('max_colwidth',200)
import matplotlib.pyplot as plt

FILE_PATH = '/home/eiao/pythoncode/StudyShare/FaceReco'

# data loading

In [3]:

num_emotions = 3   # number of out emotion
img_size = 256

def dataLoad(img_list, label_list, num_emotions=3, height=28, width=28, V2=True):
    # image: path to array
    # label: str to array
    num = len(img_list)
    all_img = np.zeros((num, height, width, 1))
    all_label = np.zeros((num, num_emotions))
    for i in range(num):
        img = cv2.imread(img_list[i], 0)
        img = cv2.resize(img, (height,width)).astype(np.float32)
        img = img / 255.0
        if V2:
            img = (img - 0.5) * 2.0
        all_img[i,:,:,0] = img
        all_label[i,:] = np.asarray((label_list[i])[2:-1].split(','))
    return all_img, all_label


label_file_path = os.path.join(FILE_PATH, 'datasets/data.csv')
data_df = pd.read_csv(label_file_path)
data_df['image'] = data_df['image'].apply(lambda x: os.path.join(FILE_PATH, 'datasets/image', x))
# data_df = data_df.sample(frac=0.1, replace=True)

train_data,test_data,train_targets,test_targets = train_test_split(data_df['image'].values,data_df['labels'].values,test_size=0.2)
train_data,val_data,train_targets,val_targets = train_test_split(train_data,train_targets,test_size=0.2)
print(len(train_targets), len(val_targets), len(test_targets))


train_x, train_y = dataLoad(train_data, train_targets, num_emotions=num_emotions, height=img_size, width=img_size, V2=True)
val_x, val_y = dataLoad(val_data, val_targets, num_emotions=num_emotions, height=img_size, width=img_size, V2=True)
test_x, test_y = dataLoad(test_data, test_targets, num_emotions=num_emotions, height=img_size, width=img_size, V2=True)

704 176 220


# model training

In [29]:
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

from models.test_model import mini_XCEPTION

# parameters
batch_size = 4
num_epochs = 3
input_shape = (img_size, img_size, 1)
verbose = 1

patience = 50
base_path = os.path.join(FILE_PATH, 'trained_models')

# data generator
data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=False)

# model parameters/compilation
model = mini_XCEPTION(input_shape, num_emotions)
model.compile(optimizer='adam', loss='mse')
model.summary()


# callbacks
log_file_path = os.path.join(base_path,'_emotion_training.log')
csv_logger = CSVLogger(log_file_path, append=False)
early_stop = EarlyStopping('val_loss', patience=patience)
reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1, patience=int(patience/4), verbose=1)
model_names = os.path.join(base_path, '_mini_XCEPTION_{epoch:02d}.hdf5')
model_checkpoint = ModelCheckpoint(model_names, 'val_loss', verbose=1, save_best_only=True)
callbacks = [model_checkpoint, csv_logger, early_stop, reduce_lr]

# model fit

model.fit_generator(data_generator.flow(train_x, train_y,
                                        batch_size),
                    steps_per_epoch=len(train_x) / batch_size,
                    epochs=num_epochs, verbose=1, callbacks=callbacks,
                    validation_data=(val_x, val_y))

score = model.evaluate(test_x, test_y, batch_size=batch_size)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_36 (Conv2D)              (None, 254, 254, 8)  72          input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_71 (BatchNo (None, 254, 254, 8)  32          conv2d_36[0][0]                  
__________________________________________________________________________________________________
activation_31 (Activation)      (None, 254, 254, 8)  0           batch_normalization_71[0][0]     
__________________________________________________________________________________________________
conv2d_37 

Epoch 1/30
 4/44 [=>............................] - ETA: 19:15 - loss: 1.6810

KeyboardInterrupt: 

In [22]:
score = model.evaluate(test_x, test_y, batch_size=batch_size)
score

220/220 [==============================] - 18s 83ms/step


0.9041195583614436

In [23]:
score = model.evaluate(val_x, val_y, batch_size=batch_size)
score

176/176 [==============================] - 15s 83ms/step


0.97677121383392

In [24]:
emotions = model.predict(test_x, batch_size=batch_size)
emotions

array([[0.29249957, 1.4951167 , 1.832237  ],
       [0.30739826, 1.4942718 , 1.803678  ],
       [0.30609393, 1.4594591 , 1.820226  ],
       [0.29795006, 1.4776753 , 1.8156567 ],
       [0.39889458, 1.8176898 , 1.6204807 ],
       [0.32584167, 1.5457534 , 1.4278563 ],
       [0.28940213, 1.4523591 , 1.800651  ],
       [0.29447654, 1.4615823 , 1.6195652 ],
       [0.2953256 , 1.5471742 , 1.7212932 ],
       [0.30626008, 1.4746546 , 1.8252991 ],
       [0.26063794, 1.3984848 , 1.6250513 ],
       [0.3032226 , 1.4556172 , 1.823037  ],
       [0.34001714, 1.5332137 , 1.6312652 ],
       [0.2946477 , 1.4426903 , 1.7863046 ],
       [0.30355117, 1.4411235 , 1.8116202 ],
       [0.4021537 , 1.8429725 , 1.315437  ],
       [0.347598  , 1.6664913 , 1.7122136 ],
       [0.27078268, 1.454285  , 1.6658734 ],
       [0.3998894 , 1.7062552 , 1.4721189 ],
       [0.3583171 , 1.6282601 , 1.5994477 ],
       [0.2930316 , 1.4091312 , 1.8046381 ],
       [0.3000719 , 1.5029023 , 1.8160722 ],
       [0.

In [26]:
# pre = map(lambda x: x[0], emotions)
pre = pd.DataFrame(emotions,columns=['predict_smile', 'predict_sad', 'prrdict_angry'])
real = pd.DataFrame(test_y, columns=['real_smile', 'real_sad', 'real_angry'])
result = pd.concat([pre, real], axis=1)
result

,predict_smile,predict_sad,prrdict_angry,real_smile,real_sad,real_angry
0,0.292500,1.495117,1.832237,0.80,0.20,0.20
1,0.307398,1.494272,1.803678,0.00,0.60,0.80
2,0.306094,1.459459,1.820226,0.40,1.00,1.80
3,0.297950,1.477675,1.815657,0.00,3.20,1.60
4,0.398895,1.817690,1.620481,0.25,0.00,0.00
5,0.325842,1.545753,1.427856,0.60,1.60,0.40
6,0.289402,1.452359,1.800651,0.00,5.00,2.20
7,0.294477,1.461582,1.619565,0.60,2.20,2.80
8,0.295326,1.547174,1.721293,0.75,0.00,0.00
9,0.306260,1.474655,1.825299,0.50,0.50,0.00


# model building

In [ ]:
from models.resnext_101_64x4d import resnext_101_64x4d
from models.model import ResnextLogo
from torch import nn
from torch import optim

IMG_MODEL_PATH = os.path.join(FILE_PATH, 'trained_model')   # save trained model at this path

model = resnext_101_64x4d
model.load_state_dict(torch.load(os.path.join(FILE_PATH, 'models/resnext_101_64x4d.pth')))
model = ResnextLogo(model)

for param in model.parameters():
    param.require_grad = True
for param in list(model.children())[0].parameters():
    param.require_grad = False
# model.cuda()
model_name = '_'.join([model.__class__.__name__.lower(), '101', '64*4d', str(int(time.time()))])
print(model_name)

# Resnet101

In [ ]:
metrics_df = pd.DataFrame(metrics)
import matplotlib.pyplot as plt
metrics_df[['train_prec1', 'val_prec1']].plot(xlim=(0, len(metrics_df)), ylim=(0, 100))
metrics_df[['train_loss', 'val_loss']].plot(xlim=(0, len(metrics_df)))
plt.show()

In [ ]:
metrics_df

# Resnet50

In [ ]:
metrics_df = pd.DataFrame(metrics)
import matplotlib.pyplot as plt
metrics_df[['train_prec1', 'val_prec1']].plot(xlim=(0, len(metrics_df)), ylim=(0, 100))
metrics_df[['train_loss', 'val_loss']].plot(xlim=(0, len(metrics_df)))
plt.show()

In [ ]:
metrics_df

In [ ]:
metrics_df.loc[metrics_df['val_prec1'].idxmax()]